# Predicting Churn at Telco

by Kristofer Rivera
2022-03-14

## Project Overview

### Project Objectives
- The goal of this project is to find drivers of churn at Telco and answer the question: "Why are customers churning? 
- Construct a Machine Learning classification model that can accurately predict customer churn.
- Deliver a report that clearly lays out findings and insights, and summarizes the process taken to reach them.
- Present recommendations to stakeholders.


### Project Description

Telco, a large telecommunications company, is churning customers at a rate of about 26% percent. They want to find out why customers are churning and what changes they can make to reduce customer churn in the future. The minimum viable product is a machine learning model that can accurately predict customer churn at a rate above the baseline and can be used to make recommendations for potential changes.


## Initial Questions

- Is there a service type more associated with churn than expected?
- Is a certain payment or contract type associated with higher churn?
- Do customers who churn have higher average monthly charges?
- Do monthly charges increase with tenure? How does this affect churn?


## Wrangle Titanic Data

To acquire the telco data, I used the telco_churn database in our mySQL server. I selected all columns from the customers table and joined with the contract_types, payment_types, and internet_service types tables.

`SELECT * FROM customers
        JOIN contract_types USING (contract_type_id)
        JOIN payment_types USING (payment_type_id)
        JOIN internet_service_types USING (internet_service_type_id);`. 

In [1]:
# acquire data from sql database 
import acquire as aq
 
# make the connection the database and run the query and return the table in the form of a dataframe. 
telco = aq.get_telco_data()

Using cached csv...


To clean the data, I did the following: 

1. Dummy Vars: Get dummy variables for sex, embarked
1. Drop columns: embarked and sex columns (now dummy vars), deck (almost 700 missing values), class (class and pclass represented the same information, so I kept the one represented as an integer). 
1. Drop rows: where age is missing (177 rows are missing values, and age will likely be a very influential feature). 

In [2]:
import prepTitanic
import pandas as pd

train, validate, test = prepTitanic.clean_split_titanic_data(df)

- Our data is now split into 3 samples so that we can use the train sample to explore, fit to models and evaluate without looking at new data. 
- The validate is used to verify that our models built are not overfit to the train dataset. 
- The test set is used to give an estimate of how this model will perform on data is has never seen in the future. 

## Set the Data Context

x% of passengers survived. We have data from 712 passengers, x of which are males, x of which traveled in 1st class, and the average fare of all passengers was $x.yy. 

In [ ]:
# build a viz here setting the context of the data 

## Exploratory Analysis: Who was more likely to survive? 

I have answered these questions

1. 
2. 
3.
4. 

### Question 1

Was the phrase "women and children first" just a myth or did they really try to get out women and children first? 

  - are women more likely to survive? plot barplot x-axis is sex and y-axis is survival rate
  - are children more likely to survive? bin age into 0-16, 17+, plot barplot on x-axis where y is survival rate (new variable = is_child)
  - run chi-square test sex + survival
  - run a chi-square test is_child + survival
  - run a t-test on age and survived

### Answer 1

Women are more likely to survive.

### Question 2

...

### Answer 4

### Summary: So, who IS most likely to survive? 


*features you will more forward with*

## Predicting Survival

Because survival is a boolean/yes or no value, we will use classification machine learning algorithms to fit to our training data and we will evaluate on validate. We will pick the best model using accuracy <evaluation metric> because the data is mostly balanced, and the risk of an event happening balances the risk of getting a prediction wrong. 
    
### Baseline

I will set a baseline prediction by predicting all passengers to not survive (survived == 0). I will evaluate the accuracy of that. 

In [8]:
# baseline

baseline performed at x% accuracy

### Fit 3 models

The top 3 models we built were a decision tree, random forest, and knn. 
The models differed in (the features, the algorithms)
The random forest performed the best at x% accuracy as you will see. 

In [9]:
# fit on train
# predict on train
# evaluate on train

In [10]:
# predict on validate
# evaluate on validate

None of these model appear to be overfit. X model performed the best on the validate sample, therefore, I moved forward with that model. 

In [11]:
# predict on test
# evaluate on test

I expect this model to perform with x% accuracy in the future on data it has not seen, given no major changes in our data source. 

## Conclusion

### Summary

### Recommendations

### Next Steps